In [2]:
import pandas as pd # Need Pandas, yo!

## Combing Multiple CSV Files

In [3]:
### Combining all the NYSE .csv files
import glob
    # module to used to retrieve files/pathnames files, https://docs.python.org/3/library/glob.html

source_files = sorted(glob.glob('data/NYSE/*.csv'))
  # .glob('yourpath\*.csv') generates a list containing the file names
  # sorted() arranges the list in alphabetical order

### Iterate through source_files to combine into one df
dataframe_list = [] # generating a list containing the df from each file
for file in source_files:
    df = pd.read_csv(file)
    dataframe_list.append(df)

# Concatenate the list, ie combine into one 
df_NYSE = pd.concat(dataframe_list)

In [ ]:
df_NYSE

,Symbol,Date,Open,High,Low,Close,Volume
0,A,02 Jan 2023,149.6500,149.6500,149.6500,149.6500,0
1,AA,02 Jan 2023,45.4700,45.4700,45.4700,45.4700,0
2,AAC,02 Jan 2023,10.0700,10.0700,10.0700,10.0700,0
3,AAC.U,02 Jan 2023,10.1700,10.1700,10.1700,10.1700,0
4,AAC.W,02 Jan 2023,0.4663,0.4663,0.4663,0.4663,0
...,...,...,...,...,...,...,...
2955,ZTR,20 Oct 2023,4.7700,4.7800,4.7000,4.7200,365200
2956,ZTS,20 Oct 2023,167.6200,168.2600,165.0900,167.0900,1932000
2957,ZUO,20 Oct 2023,7.8500,7.8500,7.4200,7.6300,631300
2958,ZVIA,20 Oct 2023,1.9500,1.9900,1.8808,1.9300,140600


# Outline
1. Start with one stock and get the data set aligned with VOO.
-the dates need to aline
-calculate the percent change

2. run correlation

3. Then move to iterate through complete stock exchange to find stocks that correlate

## Acheive Correlation with One Stock

Determine stocks that have a positive correlation with VOO to predict price increases
-Based on percent change from the previous day

Then see if when the group of stocks has a positive gain does that translate into a postive gain the next day for VOO

### Selecting a stock from NYSE

In [ ]:
### Select a stock from NYSE dataset
# Berkshire Hathaway
df_BRK = df_NYSE[df_NYSE['Symbol'] == 'BRK.A']
df_BRK.head(1)

,Symbol,Date,Open,High,Low,Close,Volume
461,BRK.A,02 Jan 2023,468711.0,468711.0,468711.0,468711.0,0


### Creating VOO dataframe

In [ ]:
### Reading the VOO dataset
df_VOO = pd.read_csv('VOO Data/VOO_20100909_20231227.csv')

# Quick look at the dataset
df_VOO

,Symbol,Date,Open,High,Low,Close,Volume
0,VOO,09 Sep 2010,102.50,102.500,101.1400,101.32,53000
1,VOO,10 Sep 2010,101.68,101.860,101.3000,101.78,17200
2,VOO,13 Sep 2010,102.96,103.140,102.5000,103.06,67500
3,VOO,14 Sep 2010,102.84,103.480,102.3800,103.04,118800
4,VOO,15 Sep 2010,102.62,103.380,102.4000,103.30,18500
...,...,...,...,...,...,...,...
3460,VOO,20 Dec 2023,435.51,437.305,429.9000,430.09,5783200
3461,VOO,21 Dec 2023,433.13,434.640,430.8552,434.28,4621600
3462,VOO,22 Dec 2023,435.46,436.838,433.4800,435.29,4037800
3463,VOO,26 Dec 2023,435.67,437.920,435.5800,437.10,3835800


### Adding percent change to both dfs

In [ ]:
### Add column to df_BRK for the percent change

# Test to understand .iloc
print(df_BRK.iloc[0,5]) # syntax [row,column]

# Loop to calculate the percent of change
i = 0
percent_gain = []
while i < len(df_BRK['Close'])-1: # iterating over 'close' column, have to go -1 the length otherwise the loop goes out of 'bounds'
  percent_gain.append((df_BRK.iloc[i+1,5]-df_BRK.iloc[i,5])/df_BRK.iloc[i,5]) # percent change formula (x2-x1)/x1
  i+=1

# adding 0 at the begining of the list
# recall percent change is not possible for the first date in the df
percent_gain.insert(0,0)
print(len(percent_gain))

# Add the percent_gain list to the df
df_BRK['Gain'] = percent_gain
df_BRK


468711.0
207


,Symbol,Date,Open,High,Low,Close,Volume,Gain
461,BRK.A,02 Jan 2023,468711.0,468711.0,468711.0,468711.0,0,0.000000
461,BRK.A,03 Jan 2023,472950.0,473990.0,465019.8,469525.0,5200,0.001737
461,BRK.A,04 Jan 2023,477479.0,479897.1,470151.0,476259.9,3700,0.014344
461,BRK.A,05 Jan 2023,481229.0,481229.0,469227.5,473660.0,3400,-0.005459
461,BRK.A,06 Jan 2023,475989.8,484150.6,473300.0,481955.0,3800,0.017513
...,...,...,...,...,...,...,...,...
452,BRK.A,16 Oct 2023,527957.5,531300.0,525380.0,527475.1,9100,0.009135
460,BRK.A,17 Oct 2023,525924.0,530475.1,524000.0,527750.0,7000,0.000521
451,BRK.A,18 Oct 2023,526115.5,528030.5,517200.0,518145.0,7600,-0.018200
454,BRK.A,19 Oct 2023,518292.5,520674.5,514425.1,515550.0,6900,-0.005008


In [ ]:
### Add column to df_VOO for the percent change

# Test to understand .iloc
print(df_VOO.iloc[0,5]) # syntax [row,column]

# Loop to calculate the percent of change
i = 0
percent_gain = []
while i < len(df_VOO['Close'])-1: # iterating over 'close' column, have to go -1 the length otherwise the loop goes out of 'bounds'
  percent_gain.append((df_VOO.iloc[i+1,5]-df_VOO.iloc[i,5])/df_VOO.iloc[i,5]) # percent change formula (x2-x1)/x1
  i+=1

# adding 0 at the begining of the list
# recall percent change is not possible for the first date in the df
percent_gain.insert(0,0)
print(len(percent_gain))

# Add the percent_gain list to the df
df_VOO['Gain'] = percent_gain
df_VOO


101.32
3465


,Symbol,Date,Open,High,Low,Close,Volume,Gain
0,VOO,09 Sep 2010,102.50,102.500,101.1400,101.32,53000,0.000000
1,VOO,10 Sep 2010,101.68,101.860,101.3000,101.78,17200,0.004540
2,VOO,13 Sep 2010,102.96,103.140,102.5000,103.06,67500,0.012576
3,VOO,14 Sep 2010,102.84,103.480,102.3800,103.04,118800,-0.000194
4,VOO,15 Sep 2010,102.62,103.380,102.4000,103.30,18500,0.002523
...,...,...,...,...,...,...,...,...
3460,VOO,20 Dec 2023,435.51,437.305,429.9000,430.09,5783200,-0.018485
3461,VOO,21 Dec 2023,433.13,434.640,430.8552,434.28,4621600,0.009742
3462,VOO,22 Dec 2023,435.46,436.838,433.4800,435.29,4037800,0.002326
3463,VOO,26 Dec 2023,435.67,437.920,435.5800,437.10,3835800,0.004158


### Run correlation on percent change

In [ ]:
### Aligning dates of the DFs
'''
-Use the data range of the stock to select the data range of VOO
-Recall first date does not have a percent change
-Intent is to have a leading indicator for VOO, so
'''

df_BRK_PC = df_BRK[['Date','Gain']]
df_BRK_PC

df_VOO_PC = df_VOO[['Date','Gain']]

## Merging the two dataframes
# already truncates VOO to the date range of the stock
result_df = pd.merge(df_BRK_PC, df_VOO_PC, on='Date', how='inner')
result_df



,Date,Gain_x,Gain_y
0,02 Jan 2023,0.000000,0.000000
1,03 Jan 2023,0.001737,-0.003842
2,04 Jan 2023,0.014344,0.007200
3,05 Jan 2023,-0.005459,-0.010922
4,06 Jan 2023,0.017513,0.022744
...,...,...,...
202,16 Oct 2023,0.009135,0.010469
203,17 Oct 2023,0.000521,0.000025
204,18 Oct 2023,-0.018200,-0.013381
205,19 Oct 2023,-0.005008,-0.008476


In [ ]:
# Test is understnad why VOO is 0% gain on 02 Jan
print(df_VOO[df_VOO['Date'] == '02 Jan 2023'])
print(df_VOO.iloc[3211,])

     Symbol         Date    Open    High     Low   Close  Volume  Gain
3212    VOO  02 Jan 2023  351.34  351.34  351.34  351.34       0   0.0
Symbol            VOO
Date      30 Dec 2022
Open           349.79
High           351.49
Low          347.7635
Close          351.34
Volume        5325400
Gain        -0.002753
Name: 3211, dtype: object


In [ ]:
# Test to see if data is aligned by date
print(df_VOO[df_VOO['Date'] == '02 Jan 2023'])

     Symbol         Date    Open    High     Low   Close  Volume  Gain
3212    VOO  02 Jan 2023  351.34  351.34  351.34  351.34       0   0.0


In [ ]:
### Shift VOO
'''
Need to shift VOO pc down one. The intent is to have a leading indicator for VOO.
So performance on the stock the day before predicts the next day on VOO
'''
#remove date column
df1 = result_df.drop(['Date'], axis=1)

# shifting VOO
df1['Gain_y'] = df1.Gain_y.shift(1,fill_value=0) # Shifting VOO
  # resource https://www.tutorialspoint.com/how-to-shift-a-column-in-a-pandas-dataframe
  # resource https://towardsdatascience.com/all-the-pandas-shift-you-should-know-for-data-analysis-791c1692b5e

# Removing the first two rows and the last
df2 = df1.drop([0,1,len(df1['Gain_y'])-1])
df2




,Gain_x,Gain_y
2,0.014344,-0.003842
3,-0.005459,0.007200
4,0.017513,-0.010922
5,-0.007408,0.022744
6,0.002164,-0.000729
...,...,...
201,-0.003464,-0.006011
202,0.009135,-0.005269
203,0.000521,0.010469
204,-0.018200,0.000025


In [ ]:
### Find correlation
df2.corr(method='pearson')

,Gain_x,Gain_y
Gain_x,1.000000,-0.082098
Gain_y,-0.082098,1.000000
